In [1]:
import numpy as np
from scipy import stats

### Forward algorithm

Given the set of observation $\textbf{X}^{(T)}=(X_1, X_2 ... X_T)$ and HMM model parameters $\theta$ the likelihood of the observations $\textbf{x}^{(T)}$ is given by

$$ L_T = P(\textbf{X}^{(T)} = \textbf{x}^{(T)} | \theta) = \boldsymbol{\delta} \boldsymbol{P}(x_1) \boldsymbol{\Gamma} \boldsymbol{P}(x_2) ... \boldsymbol{\Gamma} \boldsymbol{P}(x_T) $$

where $\boldsymbol\delta$ is the 1 X N the vector containing the inital probability distribution, $\boldsymbol\Gamma$ is the N X N matrix of transition probabilities and $\boldsymbol P (x)$ is a N X N diagonal matrix of state-dependent densities.

The likelihood can be effciently computed using dynamic programming. Defining the 1 X N probability vector
$$\boldsymbol \alpha_t = \boldsymbol{\delta} \boldsymbol{P}(x_1) \boldsymbol{\Gamma} \boldsymbol{P}(x_2) ... \boldsymbol{\Gamma} \boldsymbol{P}(x_t) $$

Which can be calculated using the recursion
$$ \boldsymbol \alpha_1 = \boldsymbol \delta \boldsymbol P(x_1)  $$
$$ \boldsymbol \alpha_t = \boldsymbol \alpha_{t-1} \boldsymbol \Gamma \boldsymbol P(x_t) \quad \text{for t = 2,3...T} $$ 
$$ L_T = \boldsymbol \alpha_T \boldsymbol 1' $$


### Pseudo code for computation of the log of scaled forward probabiltities

Let $\boldsymbol{\delta, \Gamma, P(x)}$ be defined as above. Further, let $\boldsymbol\phi$ and $w$ be vectors of length N (where N is the number of hidden states), $llk$ is a scalar to store log likelihood.

Init:
$$\boldsymbol \alpha_0 = \boldsymbol \delta \boldsymbol P(x_0)$$
$$ w_0 = \boldsymbol \alpha_0 \boldsymbol 1 $$
$$ llk = log(w_0) $$
$$ \boldsymbol \phi_0 = llk + log(\frac{\boldsymbol \alpha_0}{w_0}) $$


For t = 2,3..,T:
$$\boldsymbol \alpha_t = \frac{\boldsymbol \alpha_{t-1}}{w_{t-1}} \boldsymbol \Gamma \boldsymbol P(x_t)$$
$$ w_t = \boldsymbol \alpha_t \boldsymbol 1 $$
$$ llk = log(w_t) $$
$$ \boldsymbol \phi_t = llk + log(\frac{\boldsymbol \alpha_t}{w_t}) $$


Return $llk$ and $\boldsymbol \phi$


### JUMP model theta derivatives

The L2 norm is defined as
$$ ||\textbf{x}||_2^2 = (\sqrt{\sum_i x_i^2})^2 = \sum_i x_i^2 = \textbf{x'x} $$

where $\textbf{x}$ is a vector.
Using that, then the loss function in a jump model is

$$ l(z_t, \theta_{st}) = ||z_t-\theta_{st}||_2^2 = (z_t-\theta_{st})'(z_t-\theta_{st}) $$

And the partial derivative of $\sum_{t=1}^{T} ((z_t-\theta_{st})'(z_t-\theta_{st})$

$$ \frac{\delta \sum_{t=1}^{T} ((z_t-\theta_{st})'(z_t-\theta_{st}) }{\theta_j} $$
$$ = \frac {\sum_{t=1}^T (z_t'z_t - 2z_t'\theta_{st}+ \theta_{st}'\theta_{st} )}{\theta_j} $$
$$ = \sum_{t:s_t=j} ( 2\theta_{st}' - 2z_t' ) $$

Thus the partial derivative for $\theta_j$ is the sum of all t where the state $s_t=j$ of the vector $theta_{st}$ and $z_t$

Setting that equal to zero yields the solution:

$$ \theta_j = \frac{1}{N_j} \sum_{t:s_t=j} z_t $$

where $N_j$ is the number of elements in $s_t=j$

### Forward algorithm scaling explained
See p. 48 in Zucchini

Let $\alpha_t = (\alpha_t(1)..\alpha_t(m))$ be the forward probability vector at time t with m denoting the amount
of states. Then the likelihood can be calculated recursively as:

For $t=0$:

- $\alpha_{0} = \delta \cdot P(x_0)$
- $w_0 = \sum_{j=1}^m \alpha_0(j) =  \alpha_0 \cdot \textbf{1'} $
- $$\phi_0 = \frac{\alpha_0}{w_0}  $$
- $$log L_0 = log(w_0)  $$
- $$ log (\alpha_0) = logL_0 + log(\phi_0) $$

For $t\in 1..T$.
- $\alpha_t = \phi_{t-1} \cdot \Gamma \cdot P(x_t)$
- $w_t = \sum_{j=1}^m \alpha_t(j) =  \alpha_t \cdot \textbf{1'} $
- $$\phi_t = \frac{\alpha_t}{w_t}  $$
- $$log L_t = L_{t-1} +  log(w_t)  $$
- $$ log (\alpha_t) = logL_t + log(\phi_t) $$

From this it is clear that:

$logL_T = \sum_{t=1}^T log(w_t) = \sum_{t=1}^T log(\alpha_t\cdot \textbf{1}^T) =
 \sum_{t=1}^T log(\phi_{t-1} \cdot \Gamma \cdot P(x_t)\cdot \textbf{1}^T) $

 Thus the log likelihood follows directly from p. 48 in Zucchini.

 TODO $\alpha_T$ however doesn't seem to match as:

$log(\alpha_T) = \sum_{t=0}^T [log(w_t)] + log(\frac{\alpha_T}{w_T}) $

Since $log(x)+log(y)=log(x*y)$ then:

$log(\alpha_T) = log(w_0*w_1*...*w_{t-1}*w_T*\frac{\alpha_T}{w_T}) $

$log(\alpha_T) = log(w_0*w_1*...*w_{t-1}*\alpha_T) $

$\alpha_T = \prod_{t=0}^T (\alpha_t\cdot \textbf{1}) * \alpha_T  $

WHICH DOESn't MATCH P. 48 !!

### Rewriting $f_{jk} = \hat v_{jk}(t) $
To make the code more efficient

From the definition on p. 71 we have

$f_{jk} = \sum_{t=2}^T \hat v_{jk}(t)$

$ \hat v_{jk}(t) = \gamma_{jk}\alpha_{t-1}(j)p_k(x_t)\beta_t(k)/L_T $

Since $\gamma_{jk}$ doesn't depend on t we can move it outside expression in $f_{jk}$

$f_{jk} = \gamma_{jk} \sum_{t=2}^T \alpha_{t-1}(j)p_k(x_t)\beta_t(k)/L_T $

We can then denote $\alpha_j$, $\beta_k$ and $P_k(x)$ as 1 x T vectors containing all time-dependent values given the state j and k to obtain:
$f_{jk} = \gamma_{jk} \sum_{jk} \alpha_{j}P_k(x)\beta_k/L_T $

Which now only only requires looping through possible values of j and k.

In [4]:
from scipy import stats

obs = [0.1, 0.3, 0.2, 1]
print(obs)
print(stats.norm.pdf(obs, loc=0, scale=0.5, ))
print(stats.norm.pdf(obs, loc=1, scale=3, ))


#print(stats.norm.pdf(0, loc=[0,1], scale=[0.5,3], ))



[0.1, 0.3, 0.2, 1]
[0.78208539 0.66644921 0.73654028 0.10798193]
[0.12712927 0.12940956 0.12833562 0.13298076]


AttributeError: 'list' object has no attribute 'shape'